We have used a few resources to help us make this algorithm,

https://arpitbhayani.me/blogs/genetic-knapsack/

https://medium.com/koderunners/genetic-algorithm-part-3-knapsack-problem-b59035ddd1d6

https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python

You will find that none of the code has been copied from anywhere, these resources were used to help us only.

In [236]:
from random import choice, randrange, random
import numpy as np

In [247]:
n = 25 # number of items
max_val = 4 # modular param for item val
max_wgt = 10 # param for item weight
wgt_limit = 50 # set limit for the sack (weight limit)
solutions = 500

In [248]:
items = {} # dict of items, each has weight and value (dict of lists)

val = [i for i in range(max_wgt + 1)]
wgt = [i for i in range(max_val + 1)]

def construct_pop(n, max_val, max_wgt):
    m = [i for i in range(max_val)] # list of possible values
    p = [i for i in range(max_wgt)] # list of possible weights

    item_val = [randrange(1, len(m) + 1) for i in range(n)] # random list of val for n items
    item_wgt = [randrange(1, len(p) + 1) for i in range(n)] # random list of weights ..

    item_vw = [(val, wgt) for val, wgt in zip(item_val, item_wgt)] # make tuple (x, y) of val and weight

    for idx, i in enumerate(item_vw): # get index and populate dict 'items'
        items[idx] = i # item i = (val, wgt)

    print("List of items (item number : (weight, value))")
    print(f"Number of items: {n}")
    print(f"Maximum weight: {max_wgt}")
    print(f"Maximum value: {max_val}")
    print(items) # check if everything went as intended (spoilers it did)

    return(items)

In [249]:

pop_0 = construct_pop(n, max_val = max_val, max_wgt = max_wgt)

List of items (item number : (weight, value))
Number of items: 25
Maximum weight: 10
Maximum value: 4
{0: (4, 9), 1: (2, 7), 2: (4, 6), 3: (4, 4), 4: (3, 2), 5: (1, 8), 6: (4, 3), 7: (2, 7), 8: (3, 1), 9: (4, 3), 10: (2, 4), 11: (3, 1), 12: (3, 1), 13: (3, 4), 14: (2, 5), 15: (4, 2), 16: (1, 2), 17: (2, 2), 18: (4, 5), 19: (2, 2), 20: (3, 3), 21: (2, 2), 22: (4, 9), 23: (4, 3), 24: (1, 6)}


In [250]:
def bit_strings(sol, pop_size):
    bit_string_arr = []
    for i in range(sol):
        bit_string = [randrange(0, 2) for i in range(pop_size)] # 0 or 1, randomly, length is number of items
        bit_string_arr.append(bit_string)
    bit_string_arr = np.array(bit_string_arr)

    return bit_string_arr

solutions = bit_strings(solutions, n)

In [251]:
def assign_weights(bit_strings, items, weight_limit):
    solution_score = [] # need a list for the next step. sad.

    for bit_string in bit_strings:
        total_value = 0
        total_weight = 0

        for idx, bit in enumerate(bit_string):
            if bit == 1:  # Item is selected
                total_value += items[idx][0]  # Add value of the item
                total_weight += items[idx][1]  # Add weight of the item

        # If total weight exceeds the limit, penalize the fitness
        if total_weight > weight_limit:
            total_value -= total_value  # set score to 0 if we exceed weight limit (eliminate those solutions)

        solution_score.append(total_value)

    return solution_score

sol_score = assign_weights(solutions, pop_0, wgt_limit)

In [252]:
# https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python
# the above link is what helped us implement the roulette wheel. how it works is simple, we generate some number between 0 and 1,
# then we iterate over all probabilities, add them until their sum exceeds the random number. This is the selected number.
# the fact we have also set bad solutions (weight exceeded) to 0 means they will NEVER be selected here, which is good

def roulette_prob_wheel(pop, solution_scores):
    total_score = np.sum(solution_scores) # sum of all the scores (used for probability wheel :) )
    probabilities = solution_scores / total_score # divides each score by the total, gets probability (proportion)

    survival = []
    for i in range(len(pop)):
        rand = random()
        prob_sum = 0 # we set this equal to 0, add until we exceed "rand"
    
        for idx, prob in enumerate(probabilities):
            prob_sum += prob # add prob to sum until exceed
            if prob_sum > rand:
                survival.append(pop[idx]) # appends good item to survivor list
                break

    return survival # returns 500, == original solution size, on the right track

survivors = roulette_prob_wheel(solutions, sol_score)

a = assign_weights(survivors, pop_0, wgt_limit)
print(max(a))

44


In [253]:
def crossover(sol_size, pop, cross_rate): # we will set a rate for crossovers, this way they do not happen every time.
    crossover_results = []
    parents = [(pop[i], pop[i + 1]) for i in range(0, len(pop), 2)] # gets parent pairs, done "2 by 2" -> 250 len
    cross_point = randrange(1, sol_size) # for each bit string, different crossover index (never 0 or 20)
    for idx, parent in enumerate(parents):
        rand = random() # between 0 and 1n
        child_1 = parents[idx][0]
        child_2 = parents[idx][1]
        if rand < cross_rate: # crossover only happens if below defined rate (args)
            child_1 = np.hstack((parent[0][:cross_point], parent[1][cross_point:])) # np.hstack just concats 2 vectors of 1 row
            child_2 = np.hstack((parent[1][:cross_point], parent[0][cross_point:]))
            crossover_results.append(child_1)
            crossover_results.append(child_2)
        else:
            crossover_results.append(child_1)
            crossover_results.append(child_2)

    # crossover_results = np.concatenate(crossover_results)
    return crossover_results

cross_results = crossover(20, survivors, 0.5)

a = assign_weights(cross_results, pop_0, wgt_limit)
print(max(a))

45


In [254]:
def mutate(sol_size, pop, mutate_rate):
    mutated_results = []
    for bit_string in pop:
        for idxx, bit in enumerate(bit_string):
            rand = random()
            if rand < mutate_rate:
                bit_string[idxx] = abs(bit_string[idxx] - 1) # should shift 0 to 1 and vice versa
        mutated_results.append(bit_string)

    return mutated_results

mutated_results = mutate(20, cross_results, 0.1)

a = assign_weights(mutated_results, pop_0, wgt_limit)
print(max(a))

48


In [255]:
def genetic_algorithm(iterations, pop_size, cross_rate, mutate_rate):
    best_solution = None # we need to find an optimal solution
    best_fitness = 0 # we will report the value of the best fit at the end of this loop
    solutions = bit_strings(pop_size, n) # construct some initial population

    for iteration in range(iterations):
        solution_scores = assign_weights(solutions, items, wgt_limit) # need to evaluate scores of current solutions
        survivors = roulette_prob_wheel(solutions, solution_scores) # use roulette wheel algo to select survivors
        crossover_results = crossover(n, survivors, cross_rate) # use crossover algo to fine tune the solutions
        mutated_results = mutate(n, crossover_results, mutate_rate) # use mutation algo to fine tune even more

        mutated_scores = assign_weights(mutated_results, items, wgt_limit) # get the score of the fine-tuned solutions

        max_fitness = max(mutated_scores) # max score of all fine-tuned solutions
        if max_fitness > best_fitness: # if there is a new best solution
            best_fitness = max_fitness
            best_solution = mutated_results[np.argmax(mutated_scores)] # find with index of best solution (np.argmax = index of max)

        solutions = mutated_results # the next generation will use our fine-tuned solution!

        print(f"iter {iteration + 1}, score of current best solution: {best_fitness}") # DEBUG, should add option to be verbose or not

    return best_solution, best_fitness

# set params
iterations = 100
pop_size = 500
cross_rate = 0.4
mutate_rate = 0.1

# run algo
best_solution, best_fitness = genetic_algorithm(iterations, pop_size, cross_rate, mutate_rate)

# debug info, display score of solution
print(f"best solution: {best_solution}")
print(f"score of optim solution: {best_fitness}")

iter 1, score of current best solution: 46
iter 2, score of current best solution: 49
iter 3, score of current best solution: 49
iter 4, score of current best solution: 49
iter 5, score of current best solution: 49
iter 6, score of current best solution: 49
iter 7, score of current best solution: 49
iter 8, score of current best solution: 49
iter 9, score of current best solution: 49
iter 10, score of current best solution: 49
iter 11, score of current best solution: 49
iter 12, score of current best solution: 49
iter 13, score of current best solution: 49
iter 14, score of current best solution: 49
iter 15, score of current best solution: 49
iter 16, score of current best solution: 49
iter 17, score of current best solution: 49
iter 18, score of current best solution: 49
iter 19, score of current best solution: 49
iter 20, score of current best solution: 49
iter 21, score of current best solution: 49
iter 22, score of current best solution: 49
iter 23, score of current best solution: 

In [106]:
def elite(pop, n_largest):
    elitism = []
    bound = solutions / n_largest
    checked_sols = assign_weights(mutated_results, pop_0, wgt_limit)
    top_n = np.argsort(checked_sols)[-100:]
    for i in top_n:
        elitism.append(pop[i])

    print(len(elitism))
    return elitism

elite(mutated_results, 10)

100


[array([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1]),
 array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1]),
 array([0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1]),
 array([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1]),
 array([1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]),
 array([1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1]),
 array([0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1]),
 array([0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1]),
 array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0]),
 array([1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]),
 array([0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]),
 array([1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1]),
 array([1, 0, 1, 1, 